# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv                  # .env 파일의 환경변수 로드
import os                                       # 환경변수 접근용

load_dotenv()                                   # 현재 위치의 .env를 읽어와 환경변수로 등록
api_key=os.getenv("openai_key")                 # .env의 openai_key 값을 가져옴
client = OpenAI(api_key=api_key)



In [2]:
response = client.embeddings.create(
    model= 'text-embedding-3-small',
    input= ['Hello word1', '안녕하십니까?']     # 임베딩으로 변환할 문장 리스트로 전달
)

response

CreateEmbeddingResponse(data=[Embedding(embedding=[0.012523756362497807, -0.010440380312502384, 0.021804964169859886, 0.02258818969130516, -0.03261345997452736, -0.010338560678064823, 0.026958581060171127, 0.03223751112818718, -0.04003842547535896, 0.02164831943809986, -0.017199605703353882, 0.018390106037259102, -0.014936087653040886, 0.030702393501996994, 0.024123309180140495, 0.03168925642967224, 0.011121785268187523, 0.023919669911265373, -0.004902984015643597, 0.06127947196364403, 0.03699951618909836, 0.015460847876966, -0.017920171841979027, 0.02050481177866459, 0.00758944358676672, 0.055420953780412674, -0.007867488078773022, 0.028070759028196335, 0.02998182736337185, -0.03706217557191849, 0.011121785268187523, -0.061624087393283844, -0.030827708542346954, -0.02559577114880085, -0.036153633147478104, 0.018468428403139114, -0.010565696284174919, 0.022791827097535133, -0.008075042627751827, -0.011482068337500095, -0.008920924738049507, -0.015077068470418453, 0.050784263759851456, 

In [3]:
emb_vec0 = response.data[0].embedding   # 첫번째 문장의 임베딩 벡터
emb_vec1 = response.data[1].embedding

print(len(emb_vec0))                    # 첫 번째 임베딩 벡터 차원 (길이)
print(len(emb_vec1))

1536
1536


In [4]:
# 여러 문장을 한 번에 임베딩으로 변환하고, 각 문장의 벡터를 리스트로 반환하는 함수
def text_to_embedding(texts, model='text-embedding-3-small'):
    texts = [text.replace('\n', ' ') for text in texts]             # 개행(\n)을 공백으로 치환해 임베딩 입력 정리
    response = client.embeddings.create(model=model, input=texts)   # 임베딩 생성 요청
    
    return [data.embedding for data in response.data]               # 각 문장의 임베딩 벡터만 추출해 리스트로 반환

vecs = text_to_embedding(['Helloworld', '안녕하세요'])               #  2개 문장을 임베딩으로 변환 

print(len(vecs[0])) # 첫 번째 문장 임베딩 벡터 차원(길이)
print(len(vecs[1])) 

1536
1536


## 음식리뷰 임베딩처리

https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

corpus를 임베딩 벡터로 변환하고, 이후에 유사도를 통한 검색이 가능하다.

In [5]:
import pandas as pd

# CSV 로드 (0번 컬럼은 인덱스로 사용)
review_df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)        
review_df.info()    # 컬럼 타입/결측치/행 수 등 메타 정보
review_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
dtypes: int64(2), object(4)
memory usage: 54.7+ KB


,Time,ProductId,UserId,Score,Summary,Text
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...


In [7]:
review_df = review_df[['Summary', 'Text']]  # 필요한 컬럼 2개만 사용

# Summary 와 Text를 공백 제고 후 하나의 문자열로 결합
review_df['Content'] = 'Title: ' + review_df['Summary'].str.strip() + "; Content:" + review_df['Text'].str.strip()
review_df.head()

,Summary,Text,Content
0,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content:Not pleased ...
2,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ...."
3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...
4,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content:My dog ...


In [8]:
# Content를 리스트로 변환 임베딩 생성-> Embedding 컬럼에 저장
review_df['embedding'] = text_to_embedding(review_df['Content'].tolist())
review_df.head(3)

,Summary,Text,Content,embedding
0,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[0.03558170422911644, -0.020918212831020355, -..."
1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content:Not pleased ...,"[0.010457453317940235, 0.03552861511707306, -0..."
2,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[0.0031563059892505407, 0.013021500781178474, ..."


In [9]:
embed_df = review_df['embedding'].to_frame('embedding') # Series(embedding) -> DataFrame으로 변환
embed_df.index = review_df['Content']                   # 인덱스는 Content 텍스트로 설정
embed_df

,embedding
Content,
Title: where does one start...and stop... with a treat like this; Content:Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.03558170422911644, -0.020918212831020355, -..."
"Title: Arrived in pieces; Content:Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[0.010457453317940235, 0.03552861511707306, -0..."
"Title: It isn't blanc mange, but isn't bad . . .; Content:I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[0.0031563059892505407, 0.013021500781178474, ..."
"Title: These also have SALT and it's not sea salt.; Content:I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.003678289009258151, 0.014659896492958069, ..."
Title: Happy with the product; Content:My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[0.012326869182288647, -0.05477552115917206, 0..."
...,...
Title: Delicious!; Content:I have ordered these raisins multiple times. They are always great and arrive timely. I can't go back to store bought chocolate covered raisins now! Love this product.,"[0.014920122921466827, -0.03935643658041954, -..."
Title: Good Training Treat; Content:My dog will come in from outside when I am training her and look at the cupboard waiting for her treat. When I use the clicker training method she comes because she knows she has something special.,"[-0.022720906883478165, -0.011555996723473072,..."
Title: Jamica Me Crazy Coffee; Content:Wolfgang Puck's Jamaica Me Crazy is that wonderful blend of island flavors in a coffee. Have loved it from the first time tasting. Great product.,"[-0.029741819947957993, -0.0468682162463665, -..."


### 유사도 검색
사용자가 입력한 쿼리와 유사한 리뷰를 조회할 수 있다.(벡터 서치 기반)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity


# 쿼리와 가장 유사한 리뷰를 벡터 유사도로 Top-N개 검색하는 함수
def review_vector_search(query, embed_df=embed_df, top_n=5):
    query_emb = text_to_embedding([query])                      # 사용자 쿼리를 임베딩 벡터로 변환
    
    df = embed_df.copy()                                        # 원본 보호를 위한 복사
    df['cos_sim'] = df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])

    df = df.sort_values('cos_sim', ascending=False).head(top_n) # 유사도가 높은 순으로 정렬 후 Top-N개 선택
    df = df.reset_index()                                       # 인덱스(Content)를 컬럼으로 변환
    df = df[['Content', 'cos_sim']]                             # Top-N 검색 결과 DataFrame 생성

    return df

pd.set_option('display.max_colwidth', None)                     # Content 컬럼이 잘리는것을 방지하기 위해 최대 출력 길이 제한 해제(defalut 50)
review_vector_search('delicious fruit')                         # delicious fruit과 유사한 리뷰 Top-5 검색

,Content,cos_sim
0,"Title: Delicious!; Content:For anyone who says ""I don't like fruitcake"" or anyone who's never had fruitcake and wonders what all the fuss is about, try this. (As long as you're not allergic to tree nuts or any other ingredient.) It's chock-a-block with nuts and moist fruit. I will definitely be buying more.",0.649682
1,"Title: Delicious .; Content:These plums are sweet and juicy, and the aroma is like perfume. And it doesn't hurt that they are good for you, too.",0.612499
2,"Title: Delicious!; Content:Wonderful! Deep, rich, pure black raspberry syrup! Absolutely delicious on waffles, cheesecake, ice cream, yogurt, drinks, etc. Thrilled to see that there are at least some berry syrup makers who do not feel the need to ""sour"" the flavor of perfect berry products with citric acid!",0.603577
3,"Title: These are Delicious!; Content:Great taste, right price, fabulous snack! It is the only fruit I can get my little one to eat and I can't keep my high school son out of them either. They are great pick-me-ups on the way to my daughter's soccer practice or before my early morning run. And best of all, Amazon ships these right to my door every month. No more finding the right store who carries them. I just set up the automatic recurring shipment once and it works like a charm!",0.599620
4,"Title: very delicious; Content:This is my very favorite of the truffles. It is a medium dark and very delicious. If I have one, it screams at me to have just one more! As long as they are made, I will be first in line to purchase.",0.506357


cosine_similarity([emb], query_emb)   
emb = [[0.2131123123123, ..]] => (1, d)  
query_emb = (1, d)  
cosine similiarity 계산시 [[0.955....]] => (1, 1)   
[0, 0] => 값만 뺴온다. 

In [17]:
review_vector_search('Hawaian Pizza')

,Content,cos_sim
0,Title: Simple and Authentic; Content:This is a fantastic do-it-yourself poke product. Just add sesame oil and green onion for color then enjoy your authentic Hawaiian treat!,0.466530
1,"Title: I love Maui Coffee!; Content:My first experience with Maui Coffee was bringing it back from Maui as a souvenier. I called Maui Coffee Manufacting to order more, Shipping was a bit expensive. Originally,I would bring it back from Maui for souveniers for my friends and family on the mainland. They loved it so much I called Maui coffee and ordered more. Once I realized Amazon had Maui Coffee I was so excited and now I can order at my leisure and gift it to all my friends and family. My personal favorites are the flavored blends. (: Thanks!",0.365081
2,"Title: Unbelievable! the best rice I have ever had.; Content:I hardly ever write reviews, this rice is so good that I have to write about it.<br /><br />I live in Hawaii and have never seen this Caribbean rice mix for sale in any store. I came across this while searching Amazon, so glad I gave it a try!.. ""try it"" I do not think you'll be disappointed.<br /><br />I have added the chopped up chicken as suggested but i also added small chopped onion, it is unbelievable! Another great way to make it even better is try adding fresh Mango while it is simmering or if you cant find it fresh, use canned mango nectar, substitute 1/4 cup of the water for the mango juice. Super Good!. Also instead of 2 1/2 cups water as suggested use only 2 1/4 cups water, brings out the flavor a little bit more I think (2 cups water / 1/4 cup mango nectar).",0.357683
3,"Title: stockclerk; Content:We came across these on a trip to Hawaii that a friend had asked us to get, and found that these were fantastic. So had to get some more and this was a great way to pickup without traveling back to Hawaii!!!!",0.357452
4,"Title: Heavenly Decadence! But Oh MY!! The Calories...; Content:These Cookies are beyond tasty- They are heavenly & decadent! They have a cakelike texture with the perfect amount of pumpkin spices, and a cheesecake flavored white chip that completes the experience! And, It is such a GOOD thing they are only available during the Holiday season (and I pray they keep it that way)or I'm telling you- I would be as big as the broadside of a barn. That is no joke. Just one small cookie (approx. 1 oz with a 3"" diameter) contains a whopping 130 Calories, 4.5 grams of fat (2.5 saturated) and 21 grams of carbohydrates (12 of those sugar carbs). So, just like everything else yummy in this world, this tasty tempation should be enjoyed in moderation. Thank you Pepperidge Farms for this tasty treat during the Holidays, and my waist, hips and behind would appreciate it if you keep it as a special Holiday only treat. :)",0.325917
